## The purpose of this notebook is to turn the datasets returned from the lag creator to be grouped by the hour and add the features extracted for that hour (average polarity and tweet volumes)

In [1]:
#imports
import pandas as pd
import os

In [2]:
#get data with following lag
#this can be days or hours
granularity = "days" 
lag = 7
analyser = "TextBlob"
# analyser = "vader"

In [ ]:
#prepare filename to read
folder = "./../datasets/tweets_and_prices/"+analyser+"/";
filename = folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"

In [3]:
#read data
df = pd.read_csv(filename)

/Users/jacques/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,Unnamed: 0,Unnamed: 0_x,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp,Unnamed: 0_y,Unnamed: 0.1,id,user,fullname,url,replies,likes,retweets,text,compound
0,0,3500013,6940.89,1.914169,13283.676648,6939.657834,2018-08-31 11:17:00+00:00,2625263,2625263,1.035879e+18,tr_ticker,Tr Ticker Bot,/tr_ticker/status/1035879107391832064,0.0,0.0,0.0,#Doviz\n-------------------\n#USD : 6.5438\n#E...,0.0000
1,1,3500017,6943.10,0.131577,913.339351,6941.488132,2018-08-31 11:21:00+00:00,2625262,2625262,1.035880e+18,ERIGuide,En Route Investors,/ERIGuide/status/1035880284460339201,0.0,0.0,0.0,"En Route Investors \nInvestment Store \n$1, ...",0.5696
2,2,3500024,6952.00,9.805443,68166.284109,6951.882081,2018-08-31 11:28:00+00:00,710139,710139,1.035882e+18,AarjabJain,Aarjab jain,NaN,0.0,0.0,1.0,We want our Btc back... #gbscam #amit Bhardwaj...,0.0772
3,3,3500024,6952.00,9.805443,68166.284109,6951.882081,2018-08-31 11:28:00+00:00,2625260,2625260,1.035882e+18,DarkNightCatFx,BTC Cat(OKex/Huobi/GDAX/bitfinex/bitstamp/FX/仮...,/DarkNightCatFx/status/1035881846456737793,0.0,0.0,0.0,Zaifを使うメリットは指値出しておけばシステムのバグで１００万円のビットコインがたまに４０...,0.0000
4,4,3500027,6948.76,0.094505,656.894995,6950.921135,2018-08-31 11:31:00+00:00,2625257,2625257,1.035883e+18,NowBitcoinPrice,Bitcoin Price Now,/NowBitcoinPrice/status/1035882764770451456,0.0,0.0,0.0,"Sep 01, 2018 13:30:00 UTC | 7,045.90$ | 6,068....",0.0000


In [5]:
#remove unwanted columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0_x'], axis=1)

In [7]:
#copy dataframe so that we do not have to stay reloading data after changes
df_copy = df.copy

In [8]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14950891 entries, 0 to 14950890
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Close              float64
 1   Volume_(BTC)       float64
 2   Volume_(Currency)  float64
 3   Weighted_Price     float64
 4   timestamp          object 
 5   Unnamed: 0_y       int64  
 6   Unnamed: 0.1       int64  
 7   id                 float64
 8   user               object 
 9   fullname           object 
 10  url                object 
 11  replies            float64
 12  likes              float64
 13  retweets           float64
 14  text               object 
 15  compound           float64
dtypes: float64(9), int64(2), object(5)
memory usage: 1.8+ GB


In [9]:
#get datetime
df_copy["dt"] = pd.to_datetime(df_copy['timestamp'])
df_copy = df_copy.drop(['timestamp'], axis=1)

In [10]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14950891 entries, 0 to 14950890
Data columns (total 16 columns):
 #   Column             Dtype              
---  ------             -----              
 0   Close              float64            
 1   Volume_(BTC)       float64            
 2   Volume_(Currency)  float64            
 3   Weighted_Price     float64            
 4   Unnamed: 0_y       int64              
 5   Unnamed: 0.1       int64              
 6   id                 float64            
 7   user               object             
 8   fullname           object             
 9   url                object             
 10  replies            float64            
 11  likes              float64            
 12  retweets           float64            
 13  text               object             
 14  compound           float64            
 15  dt                 datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(9), int64(2), object(4)
memory usage: 1.8+ GB


In [11]:
#floor datetimes to hour
df_copy['DateTime'] = df_copy['dt'].dt.floor('h')

In [12]:
df_copy['DateTime']

0          2018-08-31 11:00:00+00:00
1          2018-08-31 11:00:00+00:00
2          2018-08-31 11:00:00+00:00
3          2018-08-31 11:00:00+00:00
4          2018-08-31 11:00:00+00:00
                      ...           
14950886   2019-11-22 15:00:00+00:00
14950887   2019-11-22 15:00:00+00:00
14950888   2019-11-22 15:00:00+00:00
14950889   2019-11-22 15:00:00+00:00
14950890   2019-11-22 15:00:00+00:00
Name: DateTime, Length: 14950891, dtype: datetime64[ns, UTC]

In [13]:
#group tweet volumes daily
grouped_volumes = df_copy.groupby(pd.Grouper(key='dt',freq='h')).size().reset_index(name='tweet_vol')

In [14]:
#set index to datetime
grouped_volumes.index = pd.to_datetime(vdf.index)
grouped_volumes=vdf.set_index('dt')

In [15]:
#set type of volumes to float
grouped_volumes['tweet_vol'] = grouped_volumes['tweet_vol'].astype(float)

In [16]:
grouped_volumes.head()

,tweet_vol
dt,
2018-08-31 11:00:00+00:00,9.0
2018-08-31 12:00:00+00:00,35.0
2018-08-31 13:00:00+00:00,48.0
2018-08-31 14:00:00+00:00,39.0
2018-08-31 15:00:00+00:00,39.0
...,...
2018-09-08 14:00:00+00:00,34.0
2018-09-08 15:00:00+00:00,29.0
2018-09-08 16:00:00+00:00,19.0


In [19]:
#set datetime to index of dataframe
df_copy.index = pd.to_datetime(df_copy.index)

In [20]:
#group by day and take the average of values in that day
df_copy = df_copy.groupby('DateTime').agg(lambda x: x.mean())

In [21]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10754 entries, 2018-08-31 11:00:00+00:00 to 2019-11-22 15:00:00+00:00
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Close              10754 non-null  float64            
 1   Volume_(BTC)       10754 non-null  float64            
 2   Volume_(Currency)  10754 non-null  float64            
 3   Weighted_Price     10754 non-null  float64            
 4   Unnamed: 0_y       10754 non-null  float64            
 5   Unnamed: 0.1       10754 non-null  float64            
 6   id                 10754 non-null  float64            
 7   replies            10754 non-null  float64            
 8   likes              10754 non-null  float64            
 9   retweets           10754 non-null  float64            
 10  compound           10754 non-null  float64            
 11  dt                 10754 non-null  datetime64[ns, UTC]
dtyp

In [23]:
#set DateTime 
df['DateTime'] = df['dt'].dt.floor('h')

In [24]:
#get final prices of day

#before
# df_copy["Close"] = pd.Series(df.loc[df.groupby('DateTime').dt.idxmax()]["Close"]).values
#new
df_copy["Close"] = pd.Series(df_copy.loc[df_copy.groupby('DateTime').dt.idxmax()]["Close"]).values

# df2["Open"] = pd.Series(df.loc[df.groupby('DateTime').dt.idxmin()]["Open"]).values
# df2["High"] = pd.Series(df.loc[df.groupby('DateTime').Close.idxmax()]["High"]).values
# df2["Low"] = pd.Series(df.loc[df.groupby('DateTime').Close.idxmin()]["Low"]).values

10754
10754


In [25]:
df_copy.tail()

,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Unnamed: 0_y,Unnamed: 0.1,id,replies,likes,retweets,compound,dt
DateTime,,,,,,,,,,,,
2019-11-21 10:00:00+00:00,7915.08,5.841696,46280.501793,7922.948333,2.032481e+07,2.007956e+07,1.197824e+18,0.317871,1.665175,0.607017,0.055500,2019-11-21 10:28:15.399115008+00:00
2019-11-21 11:00:00+00:00,7903.99,4.598752,36346.545736,7908.334574,2.032935e+07,2.008410e+07,1.197839e+18,0.286033,2.145125,0.661871,0.089921,2019-11-21 11:26:48.538834176+00:00
2019-11-21 12:00:00+00:00,7850.26,8.123053,63911.834402,7871.305772,2.033338e+07,2.008813e+07,1.197854e+18,0.358921,2.332988,0.644710,0.085355,2019-11-21 12:28:52.951266048+00:00
2019-11-21 13:00:00+00:00,7886.39,6.215893,48817.089252,7860.276640,2.033728e+07,2.009203e+07,1.197869e+18,0.434650,2.537234,0.821935,0.079004,2019-11-21 13:28:22.355630336+00:00
2019-11-21 14:00:00+00:00,7865.79,3.962202,31183.276977,7870.965796,2.034190e+07,2.009665e+07,1.197884e+18,0.313559,2.002825,0.572505,0.072313,2019-11-21 14:28:06.926547968+00:00
2019-11-21 15:00:00+00:00,7777.00,8.067027,63012.156934,7840.156432,2.034668e+07,2.010142e+07,1.197899e+18,0.302757,2.795152,0.760219,0.079192,2019-11-21 15:27:47.452465152+00:00
2019-11-21 16:00:00+00:00,7607.21,53.361750,404001.563607,7585.736648,2.035062e+07,2.010537e+07,1.197915e+18,0.159555,1.791678,0.410019,0.097716,2019-11-21 16:27:27.097802752+00:00
2019-11-21 17:00:00+00:00,7593.28,8.862545,67460.793029,7605.734068,2.035419e+07,2.010894e+07,1.197930e+18,0.506600,2.968026,0.813728,0.101417,2019-11-21 17:27:17.266077440+00:00
2019-11-21 18:00:00+00:00,7561.81,5.096108,38574.228793,7575.801851,2.035756e+07,2.011231e+07,1.197945e+18,0.298185,1.624316,0.459522,0.085015,2019-11-21 18:27:22.316341760+00:00


In [26]:
#add tweet volume to df
df_copy['Tweet_vol'] = grouped_volumes['tweet_vol']

In [27]:
df_copy.head()

,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Unnamed: 0_y,Unnamed: 0.1,id,replies,likes,retweets,compound,dt,Tweet_vol
DateTime,,,,,,,,,,,,,
2018-08-31 11:00:00+00:00,6948.19,2.642046,18364.756758,6949.744904,2.412467e+06,2.412467e+06,1.035883e+18,0.000000,0.000000,0.111111,0.128600,2018-08-31 11:32:40+00:00,9.0
2018-08-31 12:00:00+00:00,6946.06,2.130883,14809.674894,6949.099619,3.027106e+06,3.020099e+06,1.035892e+18,0.000000,0.228571,0.057143,0.006957,2018-08-31 12:13:05.142857216+00:00,35.0
2018-08-31 13:00:00+00:00,6963.30,3.835967,26704.894458,6958.364823,3.298422e+06,3.288203e+06,1.035908e+18,5.375000,15.395833,5.375000,0.069444,2018-08-31 13:17:43.750000384+00:00,48.0
2018-08-31 14:00:00+00:00,6900.00,3.755866,25967.189232,6952.652758,2.742633e+06,2.742633e+06,1.035922e+18,0.102564,1.025641,0.487179,0.012026,2018-08-31 14:09:03.076923392+00:00,39.0
2018-08-31 15:00:00+00:00,6927.92,15.853710,109381.975503,6903.770768,3.328492e+06,3.315915e+06,1.035937e+18,7.282051,30.717949,7.666667,0.055908,2018-08-31 15:12:24.615384576+00:00,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-22 11:00:00+00:00,7070.62,54.744473,390647.872787,7139.711340,2.039890e+07,2.015365e+07,1.198201e+18,0.059896,0.562066,0.217448,0.106849,2019-11-22 11:26:56.328113664+00:00,2304.0
2019-11-22 12:00:00+00:00,7099.68,19.604379,139285.992984,7115.175404,2.040112e+07,2.015587e+07,1.198216e+18,0.100527,0.625456,0.286988,0.116118,2019-11-22 12:26:51.341703680+00:00,2467.0
2019-11-22 13:00:00+00:00,7139.79,12.872698,91719.769313,7115.869792,2.040374e+07,2.015848e+07,1.198231e+18,0.084992,0.656968,0.267994,0.125818,2019-11-22 13:27:03.721289216+00:00,2612.0


In [29]:
#remove tweet id, retweets, likes, replies and currency volume
df_copy = df_copy.drop(['id', 'retweets', 'likes', 'replies', 'Volume_(Currency)'], axis=1)

In [30]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10754 entries, 2018-08-31 11:00:00+00:00 to 2019-11-22 15:00:00+00:00
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   Close           10754 non-null  float64            
 1   Volume_(BTC)    10754 non-null  float64            
 2   Weighted_Price  10754 non-null  float64            
 3   Unnamed: 0_y    10754 non-null  float64            
 4   Unnamed: 0.1    10754 non-null  float64            
 5   compound        10754 non-null  float64            
 6   dt              10754 non-null  datetime64[ns, UTC]
 7   Tweet_vol       10754 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(7)
memory usage: 756.1 KB


In [ ]:
folder = "./../datasets/tweets_prices_volumes_sentiment/"+analyser+"/hour_datasets/"

if not os.path.exists('my_folder'):
    os.makedirs('my_folder')

In [ ]:
df_copy.to_csv(folder+"dataset_lag_"+granularity+"_"+str(lag)+".csv")